In [1]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -p numpy,sklearn,pandas
%watermark -p ipywidgets,cv2,PIL,matplotlib,plotly
%watermark -p torch,torchvision,torchaudio
%watermark -p tensorflow,tensorboard,tflite
%watermark -p onnx,onnxruntime,tensorrt,tvm
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, Markdown, HTML, Image, Javascript
from IPython.core.magic import register_line_cell_magic, register_line_magic, register_cell_magic
display(HTML('<style>.container { width:%d%% !important; }</style>' % 95))

import sys, os, io, time, random, math
import json, base64, requests, shutil
import os.path as osp
import numpy as np

def _IMPORT(x):
    try:
        x = x.strip()
        if x.startswith('https://'):
            x = x[8:]
        if not x.endswith('.py'):
            x = x + '.py'
        if x[0] == '/':
            with open(x) as fr:
                x = fr.read()
        else:
            x = x.replace('blob/main/', '').replace('blob/master/', '')
            if x.startswith('raw.githubusercontent.com'):
                uri = 'https://' + x
                x = requests.get(uri)
                if x.status_code == 200:
                    x = x.text
            elif x.startswith('github.com'):
                uri = x.replace('github.com', 'raw.githubusercontent.com')
                mod = uri.split('/')
                for s in ['main', 'master']:
                    uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[-3:])
                    x = requests.get(uri)
                    if x.status_code == 200:
                        x = x.text
                        break
            elif x.startswith('gitee.com'):
                mod = x.split('/')
                for s in ['/raw/main/', '/raw/master/']:
                    uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[3:])
                    x = requests.get(uri)
                    if x.status_code == 200:
                        x = x.text
                        break
        exec(x, globals())
    except:
        pass

def _DIR(x, dumps=True, ret=True):
    attrs = sorted([y for y in dir(x) if not y.startswith('_')])
    result = '%s: %s' % (str(type(x))[8:-2], json.dumps(attrs) if dumps else attrs)
    if ret:
        return result
    print(result)

numpy 1.19.5
sklearn 0.0
pandas 1.1.5
ipywidgets 7.6.3
cv2 4.5.3
PIL 8.3.1
matplotlib 3.3.4
plotly 5.3.0
torch 1.8.1+cu101
torchvision 0.9.1+cu101
torchaudio not installed
tensorflow 2.6.0
tensorboard 2.6.0
tflite 2.4.0
onnx 1.10.1
onnxruntime 1.8.1
tensorrt not installed
tvm not installed


In [8]:
@register_line_cell_magic
def template_writefile(line, cell):
    path = os.path.dirname(line)
    if not os.path.exists(path):
        os.makedirs(path, exist_ok=True)
    with open(line, 'w') as fw:
        fw.write(cell.format(**globals()))


## 环境准备

### QEMU and Test on ARM64

In [14]:
%%writefile /data/nb_data/erlangai/qemu.sh

sudo apt-get install -y qemu binfmt-support qemu-user-static

# TODO FAQs: write error: Invalid argument
# copy multiarch/qemu-user-static:x86_64-aarch64:/usr/bin/qemu-aarch64-static ${pwd}
docker run --rm --privileged multiarch/qemu-user-static --reset -p yes

# Test
# docker run --rm -v ${pwd}/qemu-aarch64-static:/usr/bin/qemu-aarch64-static -t arm64v8/ubuntu uname -m
docker run --rm -t arm64v8/ubuntu uname -m

Overwriting /data/nb_data/erlangai/qemu.sh


### Jetson L4T

Check tegra version On Jetson Board

```sh
# cat /etc/nv_tegra_release
```

R32 (release), REVISION: **6.1**, GCID: 27863751, BOARD: t210ref, EABI: aarch64, DATE: Mon Jul 26 19:20:30 UTC 2021


In [23]:
%%writefile /data/nb_data/erlangai/jstson_l4t.sh

# docker pull nvcr.io/nvidia/l4t-base:r32.6.1
# sudo pip3 install --pre --extra-index-url https://developer.download.nvidia.com/compute/redist/jp/v46 tensorflow
docker pull nvcr.io/nvidia/l4t-tensorflow:r32.6.1-tf2.5-py3
# docker pull nvcr.io/nvidia/l4t-ml:r32.6.1-py3
# docker pull nvcr.io/nvidia/l4t-tensorrt:r7.0.1-runtime
    
# Test
docker run --rm -t nvcr.io/nvidia/l4t-tensorflow:r32.6.1-tf2.5-py3 uname -m
    
# Run
docker run -itd --name qemu_l4t_tf --entrypoint /bin/bash \
    --network host  --runtime nvidia \
    -v /data:/data \
    -v /home/lidong/qemu-aarch64-static:/usr/bin/qemu-aarch64-static \
    nvcr.io/nvidia/l4t-tensorflow:r32.6.1-tf2.5-py3
            
# Enter
# docker exec -it qemu_l4t bash

Overwriting /data/nb_data/erlangai/jstson_l4t.sh


## FAQs

- [sh: write error: Invalid argument - Centos 7][1]
    
[1]: https://github.com/multiarch/qemu-user-static/issues/100

## References

- [Jetson Docker Containers][5]
- [Running and Building ARM Docker Containers on x86][1]
- [NVIDIA L4T is a Linux based software distribution for the NVIDIA Jetson embedded computing platform][2]
- [Jetson Docker build][3]
- [This guide provides instructions for installing TensorFlow for Jetson Platform][4]
- [NVIDIA Container Runtime on Jetson][6]

[6]: https://github.com/NVIDIA/nvidia-docker/wiki/NVIDIA-Container-Runtime-on-Jetson#building-jetson-containers-on-an-x86-workstation-using-qemu
[5]: https://ngc.nvidia.com/catalog/Containers
[4]: https://docs.nvidia.com/deeplearning/frameworks/install-tf-jetson-platform/index.html
[3]: https://github.com/dusty-nv/jetson-containers
[2]: https://ngc.nvidia.com/catalog/containers/nvidia:l4t-base
[1]: https://www.stereolabs.com/docs/docker/building-arm-container-on-x86/

## Start Emu

In [6]:
%%writefile /raceai/raspberry_pi4/run_emu.sh

WORK_DIR=/usr/rpi
IMAGE_NAME=2021-05-07-raspios-buster-armhf-lite.img
IMAGE_LOCATION=/raceai/raspberry_pi4/images

docker run -it --rm --privileged=true \
    -v ${IMAGE_LOCATION}:${WORK_DIR}/images \
    -v /raceai/:/raceai/ \
    -w $WORK_DIR ryankurte/docker-rpi-emu \
    /bin/bash -c "./run.sh images/${IMAGE_NAME}"

Overwriting /raceai/raspberry_pi4/run_emu.sh


In [ ]:
```bash

uname -m

sudo cat /etc/*-release

```

## References

- [download raspberrypi image][1]
- [docker rpi emu][2]
- [respberrypi openvino][3]
- [onnxruntime wheels][4]
- [openvino rasbian][5]
[1]: https://www.raspberrypi.org/software/operating-systems/
[2]: https://hub.docker.com/r/ryankurte/docker-rpi-emu
[3]: https://storage.openvinotoolkit.org/repositories/openvino/packages/
[4]: https://github.com/nknytk/built-onnxruntime-for-raspberrypi-linux/tree/master/wheels
[5]: https://docs.openvinotoolkit.org/latest/openvino_docs_install_guides_installing_openvino_raspbian.html

## FAQs

- [Build on RaspberryPi4b failed: requested alignment 16 is larger than 8][1]
- [树梅派 内存交换空间swap 更改][2]

[1]: https://github.com/microsoft/onnxruntime/issues/4189
[2]: https://blog.csdn.net/jiangchao3392/article/details/77984838

In [25]:
%%writefile software_setup.sh

#!/bin/bash

USER_HOME=/home/$USER
CUDA_HOME=/usr/local/cuda

PIP_INSTALL="sudo pip3 install --retries 10 --timeout 120"
APT_INSTALL="sudo apt install -y --no-install-recommends"
BOOTRC_FILE="${USER_HOME}/.bootrc'
HOST_ADDR=`who | grep "pts/0" | awk -F" " '{match($NF, /[^(].*[^)]/); print substr($NF, RSTART,RLENGTH)}'`

echo_bootrc() {
    echo "$*" | tee -a $BOOTRC_FILE
}

echo_bashrc() {
    echo "$*" | tee -a ${USER_HOME}/.bashrc
}

# on boot script
__setup_bootrc() {
    echo -e "#!/bin/bash\nsleep 10" > $BOOTRC_FILE
    chmod 777 $BOOTRC_FILE
    sudo bash -c "echo '@reboot  root  test -x ${BOOTRC_FILE} && ${BOOTRC_FILE}' >> /etc/crontab"
}

# environment
__setup_bashrc() {
    echo_bashrc "alias pip='pip3'"
    echo_bashrc "alias python='python3'"
    echo_bashrc "export PIP_INSTALL='${PIP_INSTALL}'"
    echo_bashrc "export APT_INSTALL='${APT_INSTALL}'"
    echo_bashrc "export CUDA_HOME=${CUDA_HOME}"
}

# mount.nfs host
__setup_mntnfs(){
    mkdir -p ${USER_HOME}/omega
    ${APT_INSTALL} nfs-common
    echo_bootrc "mount.nfs ${HOST_ADDR}:/blog/public ${USER_HOME}/omega"
}

# notebook
__setup_notebook() {
    ${PIP_INSTALL} jupyter jupyter_contrib_nbextensions jupyter_nbextensions_configurator
    sudo jupyter contrib nbextension install --sys-prefix
    sudo jupyter nbextensions_configurator enable
    git clone --depth 1 https://github.com.cnpmjs.org/qrsforever/jupyter_config.git
    sudo ./jupyter_config/install.sh
    echo_bootrc "su - $USER -c 'umask 0000; jupyter notebook --no-browser --notebook-dir=${USER_HOME}/omega --allow-root --ip=0.0.0.0 --port=8118 &'"
}

# cmake
__setup_cmake() {
    wget https://github.com/Kitware/CMake/releases/download/v3.21.2/cmake-3.21.2-linux-aarch64.tar.gz
    tar zxf cmake-3.21.2-linux-aarch64.tar.gz
    cp -aprf cmake-3.21.2-linux-aarch64.tar/* /usr/
}

# machine learning
__setup_ml() {
    # tensorflow
    ${APT_INSTALL} libhdf5-serial-dev hdf5-tools libhdf5-dev zlib1g-dev zip libjpeg8-dev liblapack-dev libblas-dev gfortran
    ${PIP_INSTALL} -U pip testresources setuptools==49.6.0
    ${APT_INSTALL} --pre --extra-index-url https://developer.download.nvidia.com/compute/redist/jp/v46 tensorflow
    
    # onnx2tensor
    ${APT_INSTALL} libprotobuf-dev protobuf-compiler
    git clone --depth 1 --recursive https://github.com.cnpmjs.org/onnx/onnx-tensorrt.git
    cd onnx-tensorrt && mkdir build && cd build && cmake ../ && make -j && sudo make install
    
    # pycuda, onnx_helper
    ${PIP_INSTALL} --global-option=build_ext --global-option="-I${CUDA_HOME}/include/" --global-option="-L${CUDA_HOME}/lib" pycuda
    git clone --depth 1 https://github.com.cnpmjs.org/NVIDIA/TensorRT.git
}

__setup_bootrc
__setup_bashrc
__setup_mntnfs

Overwriting software_setup.sh
